## Data Cleaning and Preperation for WHO Suicide Stats

In [54]:
# import modules
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import math
from functools import reduce
import re
import lxml
import sqlalchemy as alch
from getpass import getpass
from dotenv import load_dotenv
import os

In [55]:
# load dataset
who = pd.read_csv("data\master.csv")

In [56]:
# looking at the info of the df
who.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   country-year        27820 non-null  object 
 8   HDI for year        8364 non-null   float64
 9    gdp_for_year ($)   27820 non-null  object 
 10  gdp_per_capita ($)  27820 non-null  int64  
 11  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [57]:
# checking the size of the df
who.shape

(27820, 12)

In [58]:
# notice that there are missing values so i will remember to take them out once im done looking at the data. 
# there is also an age column which has '5-14' and 'years' which is irrelevant to have so i will try to change it to '05-14' and 
# remove the years.
who.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [59]:
# all the countries in the dataset. I will have to remove the countries that I will not be including
len(who.country.unique())

101

In [60]:
# all the years in the dataset. I will have to remove the years i will not work with which is everything except 2015-2016.
who.year.unique()

array([1987, 1988, 1989, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       1985, 1986, 1990, 1991, 2012, 2013, 2014, 2015, 2011, 2016],
      dtype=int64)

In [61]:
# seeing how many missing values i have.
who.isnull().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
country-year              0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

In [62]:
# dropping nan values in the column (make a function later)
who.drop(['HDI for year'], axis = 1, inplace = True) 

In [68]:
who.drop(['country-year'], axis = 1, inplace = True) 

In [69]:
who

,country,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+,9,348465,2.58,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,05-14,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,05-14,44,2631600,1.67,"63,067,077,179",2309,Generation Z


In [70]:
# here i am addressing the age column as i mentioned before i wanted to add the '05-14' and remove the year as it will be better
# when it comes to vizualizing
who.loc[:, 'age'] = who['age'].str.replace(' years','')
who.loc[who['age'] == '5-14', 'age'] = '05-14'

In [71]:
who['country'] = who['country'].str.replace('United States of America','United States')

In [72]:
who

,country,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+,9,348465,2.58,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,05-14,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,05-14,44,2631600,1.67,"63,067,077,179",2309,Generation Z


In [73]:
who.to_csv("data\who_suicide_statistics_cleaned.csv", index=False)